In [1]:
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
from scipy.ndimage import median_filter
import os
%matplotlib inline

# install pyproj first via pip install
os.environ['PROJ_LIB'] = "<path-to-virtualenv>/lib64/python3.6/site-packages/pyproj/proj_dir/share/proj/proj.db"

In [2]:
def merge_rasters(rasters):
    src_files = []
    for path in rasters:
        src = rasterio.open(path)
        src_files.append(src)
    
    mosaic, out_transforms = merge(src_files, method='max', resampling=rasterio.enums.Resampling.nearest)
    # Define the metadata for the output file based on one of the input files
    out_meta = src.meta.copy()

    # Update the metadata with new dimensions, transform, and compress (if desired)
    out_meta.update({"driver": "GTiff",
                    "height": mosaic.shape[1],
                    "width": mosaic.shape[2],
                    "transform": out_transforms,
                    "count": 1,
                    "nodata": 255.,
                    "compress": "lzw"})
    return mosaic, out_meta

In [3]:
# change CRS
from rasterio.warp import calculate_default_transform, reproject, Resampling
from pyproj import CRS # don't use rasterio CRS

RASTER_DIR = "..." # directory containing all the rasters to be merged

# surveys = ['SaltPointCA', 'FortRossCA', 'BodegaHeadCA', 'TomalesPointCA', 'PointReyesCA',
#            'DrakesBayCA', 'BolinasCA', 'SanFranciscoCA', 'PacificaCA', 'HalfMoonBayCA',
#            'SanGregorioCA', 'PigeonPointCA', 'ScottCreekCA', 'SantaCruzCA', 'AptosCA']

surveys = ['PointConceptionCA', 'GaviotaCA', 'RefugioBeachCA', 'CoalOilPointCA', 'SantaBarbaraCA',
           'CarpinteriaCA', 'VenturaCA', 'HuenemeCanyonCA']

# surveys = ['CapeMendocinoCA', 'EelRiverCA', 'EurekaCA', 'ArcataCA']

# surveys = ['PointBuchonCA', 'PointEsteroCA', 'MorroBayCA']

# directory should be organized as Region -> Survey Name -> <layer-name>.tif
rasters = [os.path.join(RASTER_DIR, sur, '<layer-name>.tif') for sur in surveys]
mosaic, out_meta = merge_rasters(rasters)

In [5]:
# Save to disk
with rasterio.open(os.path.join(RASTER_DIR, "<region>", "<layer-name>.tif"), 'w', **out_meta) as dst:
    dst.write(mosaic)